In [22]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
import pandas as pd

In [2]:
dframe_path = 'data/cabauw/processed.csv'

try:
    df = pd.read_csv(dframe_path, na_values='--')
except UnicodeDecodeError:
    df = pd.read_csv(dframe_path, na_values='--', compression='gzip')


df = df[(df.ustar > 0.1) & (abs(df.H) > 10) & (df.wind > 1)]
df = df[df.ds != 201603]
#df = df.sort_values(['ds', 'tt'])
df = df.sample(frac=1)  # aka shuffle

In [3]:
features = [
    'air_dens', 'surf_temp', 'spec_hum', 'rel_hum', 'press', 'rain',
    'wind', 'ustar', 'H', 'LE', 'zL_m', 'lval'
]
train_x = df[features]
train_y = df.phi_m

In [42]:
model = Sequential([
    Dense(512, activation='elu', input_shape=(len(features),)),
    Dropout(0.5),
    Dense(1024, activation='elu'),
    Dropout(0.5),
    Dense(1024, activation='elu'),
    Dropout(0.5),
    Dense(256, activation='elu'),
    Dense(32, activation='elu'),
    Dense(8, activation='elu'),
    Dense(1)
])

model.compile(loss='mse', optimizer='rmsprop')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_57 (Dense)             (None, 512)               6656      
_________________________________________________________________
dropout_10 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_58 (Dense)             (None, 1024)              525312    
_________________________________________________________________
dropout_11 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_59 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
dropout_12 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_60 (Dense)             (None, 256)               262400    
__________

In [ ]:
model.fit(train_x, train_y, batch_size=32, epochs=1, validation_split=0.25)
model.fit(train_x, train_y, batch_size=64, epochs=1, validation_split=0.25)
model.fit(train_x, train_y, batch_size=128, epochs=1, validation_split=0.25)
model.fit(train_x, train_y, batch_size=1024, epochs=10, validation_split=0.25)
model.fit(train_x, train_y, batch_size=8192, epochs=25, validation_split=0.25)

Train on 390748 samples, validate on 130250 samples
Epoch 1/1
390748/390748 [==============================] - 113s 289us/step - loss: 474012.6922 - val_loss: 0.1885
Train on 390748 samples, validate on 130250 samples
Epoch 1/1
390748/390748 [==============================] - 58s 148us/step - loss: 397759.2209 - val_loss: 0.1484
Train on 390748 samples, validate on 130250 samples
Epoch 1/1
390748/390748 [==============================] - 34s 88us/step - loss: 477051.1789 - val_loss: 0.1488
Train on 390748 samples, validate on 130250 samples
Epoch 1/10
390748/390748 [==============================] - 12s 30us/step - loss: 6459.9101 - val_loss: 0.1557
Epoch 2/10
390748/390748 [==============================] - 12s 31us/step - loss: 68.6105 - val_loss: 0.1529
Epoch 3/10
390748/390748 [==============================] - 12s 30us/step - loss: 5.5826 - val_loss: 0.1477
Epoch 4/10
390748/390748 [==============================] - 12s 31us/step - loss: 1295.8704 - val_loss: 0.1418
Epoch 5/10
390